## Check Data Correctness

In [6]:
import json
import os
from pymilvus import MilvusClient

# --- CẤU HÌNH ---
# Đường dẫn tính từ folder 'test_eval' đi ra ngoài một cấp (..) rồi vào 'inputs'
INPUT_FILE_PATH = '../inputs/gold_standard_dataset.json' 

# Cấu hình Milvus
MILVUS_URI = "http://127.0.0.1:19530"
COLLECTION_NAME = "globaltech_nlp_project"

print(f"📂 Đang kiểm tra file: {os.path.abspath(INPUT_FILE_PATH)}")

# Kiểm tra file có tồn tại không
if not os.path.exists(INPUT_FILE_PATH):
    print("❌ LỖI: Không tìm thấy file input. Hãy kiểm tra lại đường dẫn.")
else:
    print("✅ Đã tìm thấy file input.")

📂 Đang kiểm tra file: e:\My Document\GlobalTechJSC_NLP\inputs\gold_standard_dataset.json
✅ Đã tìm thấy file input.


In [7]:
def load_and_extract_ids(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        # Xử lý cả trường hợp file là JSON list hoặc JSONL
        try:
            data = json.load(f) # Dạng chuẩn [...]
        except json.JSONDecodeError:
            f.seek(0)
            data = [json.loads(line) for line in f] # Dạng JSONL

    all_ground_truth_ids = set()
    query_map = {} # Map để tra cứu ngược: doc_id này thuộc câu hỏi nào

    for idx, item in enumerate(data):
        q_id = item.get('id', str(idx))
        
        # Lấy danh sách ground_truths
        gts = item.get('ground_truths', [])
        
        for gt in gts:
            # Xử lý linh hoạt: gt có thể là dict hoặc string/int
            doc_id = str(gt['doc_id']) if isinstance(gt, dict) and 'doc_id' in gt else str(gt)
            
            all_ground_truth_ids.add(doc_id)
            
            # Lưu lại để lát nữa báo cáo xem doc_id bị thiếu thuộc câu hỏi nào
            if doc_id not in query_map:
                query_map[doc_id] = []
            query_map[doc_id].append(q_id)
            
    return list(all_ground_truth_ids), query_map, len(data)

# Thực thi
target_ids, query_map, total_queries = load_and_extract_ids(INPUT_FILE_PATH)
print(f"📊 Tổng số câu query: {total_queries}")
print(f"🔍 Tổng số doc_id duy nhất cần kiểm tra (Ground Truths): {len(target_ids)}")

📊 Tổng số câu query: 100
🔍 Tổng số doc_id duy nhất cần kiểm tra (Ground Truths): 357


In [5]:
client = MilvusClient(uri=MILVUS_URI)

def check_existence_in_milvus(doc_ids, collection_name, batch_size=1000):
    found_ids = set()
    total = len(doc_ids)
    
    print(f"🚀 Bắt đầu kiểm tra {total} IDs trong Milvus...")
    
    for i in range(0, total, batch_size):
        batch = doc_ids[i : i + batch_size]
        
        # Query Milvus xem những ID này có tồn tại không
        # Lưu ý: Trường chứa ID bài báo trong Milvus của bạn là 'original_id' hay 'id'?
        # Dựa trên code cũ của bạn là 'original_id'.
        res = client.query(
            collection_name=collection_name,
            filter=f"original_id in {batch}", 
            output_fields=["original_id"]
        )
        
        # Thu thập những ID tìm thấy
        for item in res:
            found_ids.add(str(item['original_id']))
            
    return found_ids

# Thực thi kiểm tra
found_ids = check_existence_in_milvus(target_ids, COLLECTION_NAME)
missing_ids = set(target_ids) - found_ids

print("-" * 30)
print(f"✅ Tìm thấy: {len(found_ids)} bài.")
print(f"❌ Bị thiếu (Missing): {len(missing_ids)} bài.")
print("-" * 30)

🚀 Bắt đầu kiểm tra 357 IDs trong Milvus...
------------------------------
✅ Tìm thấy: 357 bài.
❌ Bị thiếu (Missing): 0 bài.
------------------------------


In [6]:
if len(missing_ids) == 0:
    print("🎉 TUYỆT VỜI! 100% dữ liệu Ground Truth đều tồn tại trong Milvus.")
else:
    print("⚠️ CẢNH BÁO: Các câu hỏi sau đây có đáp án KHÔNG TỒN TẠI trong Database:")
    print("=" * 60)
    
    # Gom nhóm theo câu hỏi
    affected_queries = {}
    for missing in missing_ids:
        query_ids = query_map.get(missing, [])
        for q_id in query_ids:
            if q_id not in affected_queries:
                affected_queries[q_id] = []
            affected_queries[q_id].append(missing)
            
    # In ra danh sách
    count = 0
    for q_id, ids in affected_queries.items():
        count += 1
        print(f"{count}. Query ID: {q_id}")
        print(f"   - Missing Docs: {ids}")
        if count >= 20:
            print("... (và còn nhiều câu khác, chỉ hiển thị 20 câu đầu) ...")
            break

    # Tính tỷ lệ phần trăm dữ liệu hỏng
    missing_ratio = (len(missing_ids) / len(target_ids)) * 100
    print("=" * 60)
    print(f"📉 Tỷ lệ dữ liệu bị thiếu (Data Loss): {missing_ratio:.2f}%")
    print("👉 Khuyến nghị: Bạn cần kiểm tra lại quy trình Insert dữ liệu vào Milvus.")
    print("   Có thể những bài báo này bị lỗi khi crawl hoặc bị lỗi khi embedding nên chưa được nạp vào.")

🎉 TUYỆT VỜI! 100% dữ liệu Ground Truth đều tồn tại trong Milvus.


## Test Qwen: Qwen3 30B A3B Instruct 2507

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

completion = client.chat.completions.create(
    extra_body={},
    model="qwen/qwen3-30b-a3b-instruct-2507",
    messages=[
        {
            "role": "user",
            "content": "Who won the world cup in 2018?"
        }
    ]
)
print(completion.choices[0].message.content)

The winner of the 2018 FIFA World Cup was **France**. They defeated Croatia 4-2 in the final, which was held on July 15, 2018, at the Luzhniki Stadium in Moscow, Russia. This victory marked France's second World Cup title, their first since 1998. Kylian Mbappé and Antoine Griezmann were key players for France in the tournament.


## RAG

In [1]:
import os
import json
import time
import torch
import gc
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import logging as transformers_logging

load_dotenv()

# --- CONFIGURATION ---
transformers_logging.set_verbosity_error()

# Load environment variables
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY") 

# Models Config
EMBED_MODEL_NAME = "hiieu/halong_embedding"
RERANKER_MODEL_NAME = "itdainb/PhoRanker"
LLM_MODEL_NAME = "qwen/qwen3-30b-a3b-instruct-2507"

# Milvus Config
MILVUS_URI = "http://127.0.0.1:19530"
COLLECTION_NAME = "globaltech_nlp_project"

# Paths
INPUT_TEST_FILE = '../inputs/gold_standard_200.jsonl'
OUTPUT_CSV_FILE = '../outputs/rag_final_answers_200.csv'

# --- SETUP DEVICES ---
torch.cuda.empty_cache()
gc.collect()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device.upper()}")

# --- LOAD MODELS ---
print("1. Loading Embedding Model...")
embed_model = SentenceTransformer(EMBED_MODEL_NAME, device=device)

print("2. Loading Reranker Model...")
reranker = CrossEncoder(RERANKER_MODEL_NAME, max_length=256, device=device)

print("3. Connecting to Milvus...")
milvus_client = MilvusClient(uri=MILVUS_URI)
milvus_client.load_collection(COLLECTION_NAME)

print("4. Setting up LLM Client...")
llm_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

print("Setup Complete!")

e:\Anaconda\envs\LLMs-env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Device: CUDA
1. Loading Embedding Model...
2. Loading Reranker Model...
3. Connecting to Milvus...
4. Setting up LLM Client...
Setup Complete!


In [2]:
FULL_SOURCE_FILE = '../inputs/merged_news.json' 

def build_doc_lookup(file_path):
    """
    Read the full source JSON file and build a lookup dictionary {doc_id: full_text} for quick access.
    """
    lookup_dict = {}
    print(f"Building lookup dictionary from {file_path}...")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            # Handle both JSON List and JSONL formats
            try:
                data = json.load(f) # Standard JSON list
            except json.JSONDecodeError:
                f.seek(0)
                data = [json.loads(line) for line in f if line.strip()] # JSONL format
            
            for item in data:
                # Assume the key containing the ID is 'id' and the content is 'text' or 'content'
                # Adjust these keys if your JSON file uses different names
                doc_id = str(item.get('id', ''))
                content = item.get('text') or item.get('content') or ""
                
                if doc_id:
                    lookup_dict[doc_id] = content
                    
        print(f"Loaded {len(lookup_dict)} documents into memory.")
        return lookup_dict
    except Exception as e:
        print(f"Error building lookup: {e}")
        return {}
    
doc_lookup_map = build_doc_lookup(FULL_SOURCE_FILE)

Building lookup dictionary from ../inputs/merged_news.json...
Loaded 758023 documents into memory.


In [3]:
def retrieve_and_rerank_full_docs(query_text, top_k_candidates=100, final_top_k=3):
    """
    Retrieve candidate chunks from Milvus, rerank them, and map back to full documents with deduplication.
    """
    # Retrieval
    query_vector = embed_model.encode([query_text])
    search_res = milvus_client.search(
        collection_name=COLLECTION_NAME,
        data=query_vector,
        limit=top_k_candidates,
        search_params={"metric_type": "COSINE", "params": {"nprobe": 64}},
        output_fields=["title", "text", "original_id"] # Need original_id field
    )
    hits = search_res[0]
    if not hits: return []

    # Re-ranking
    # Still score based on chunk text because it is concise and easy to match the query
    cross_inp = [[query_text, hit['entity']['text']] for hit in hits]
    cross_scores = reranker.predict(cross_inp)
    
    for idx, hit in enumerate(hits):
        hit['cross_score'] = float(cross_scores[idx])
    
    # Sort
    sorted_hits = sorted(hits, key=lambda x: x['cross_score'], reverse=True)
    
    # Map to Full Document & Deduplicate
    unique_full_docs = []
    seen_ids = set()
    
    for hit in sorted_hits:
        # Get original_id from Milvus hit
        orig_id = str(hit['entity'].get('original_id'))
        
        # If this document hasn't been added and exists in the original data store
        if orig_id not in seen_ids and orig_id in doc_lookup_map:
            full_text = doc_lookup_map[orig_id]
            
            # (Optional) Add title at the beginning for better LLM understanding
            title = hit['entity'].get('title', '')
            full_context = f"Title: {title}\nContent: {full_text}"
            
            unique_full_docs.append(full_context)
            seen_ids.add(orig_id)
        
        # Stop if we have enough original documents
        if len(unique_full_docs) >= final_top_k:
            break
            
    return unique_full_docs

# --- LOGIC GENERATE ANSWER ---
def generate_answer(query, context_docs):
    if not context_docs:
        context_str = "No relevant information found."
    else:
        # Context now consists of full-length articles
        context_str = "\n\n" + "="*20 + "\n\n".join([f"Document {i+1}:\n{doc}" for i, doc in enumerate(context_docs)])

    system_prompt = """Bạn là một trợ lý AI hữu ích. Hãy trả lời câu hỏi dựa trên các tài liệu gốc được cung cấp bên dưới (Context).
Hãy tổng hợp thông tin từ các bài viết để đưa ra câu trả lời đầy đủ và chính xác nhất.
Nếu không tìm thấy thông tin, hãy nói 'Tôi không tìm thấy thông tin'."""
    
    user_prompt = f"""Context:
{context_str}

Question: 
{query}

Answer:"""

    try:
        completion = llm_client.chat.completions.create(
            extra_body={},
            model=LLM_MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating answer: {e}"

# --- RUN NEW PIPELINE ---
def run_rag_pipeline(query):
    # Retrieve Full Docs (only take top 3 original documents to avoid context overflow)
    full_context_docs = retrieve_and_rerank_full_docs(query, top_k_candidates=100, final_top_k=3)
    
    # Generate answer
    answer = generate_answer(query, full_context_docs)
    return answer

In [4]:
def load_test_data(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            print(f"📂 Loading data from: {file_path}")
            # Handle both JSON List and JSONL formats
            try:
                content = f.read()
                data = json.loads(content)
            except json.JSONDecodeError:
                f.seek(0)
                for line in f:
                    if line.strip():
                        data.append(json.loads(line))
        print(f"✅ Loaded {len(data)} test cases.")
        return data
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return []

# --- MAIN EXECUTION ---
test_cases = load_test_data(INPUT_TEST_FILE)
# test_cases = test_cases[:5] # Uncomment nếu muốn test nhanh 5 câu đầu

results = []

print(f"🚀 Starting RAG Generation for {len(test_cases)} queries...")
print("-" * 50)

for idx, case in enumerate(test_cases):
    # Lấy Query
    query = case.get('question') or case.get('query')
    
    # Lấy ID (Quan trọng để map với ground truth sau này)
    case_id = case.get('id', str(idx))
    
    if not query: 
        print(f"⚠️ Skipping Case #{case_id}: No query found.")
        continue
    
    print(f"[{idx+1}/{len(test_cases)}] Processing ID: {case_id}...", end="\r")
    
    # Run pipeline
    t0 = time.time()
    try:
        answer = run_rag_pipeline(query)
    except Exception as e:
        answer = f"Pipeline Error: {e}"
        
    latency = time.time() - t0
    
    # Lưu kết quả gồm ID, Query, Answer
    results.append({
        "id": case_id,         # <--- Cập nhật: Lưu thêm ID
        "query": query,
        "answer": answer,
        # "latency": latency   # <--- Uncomment nếu muốn lưu thời gian chạy
    })

print(f"\n✅ Completed processing {len(results)} queries.")

# --- EXPORT TO CSV ---
df = pd.DataFrame(results)
os.makedirs(os.path.dirname(OUTPUT_CSV_FILE), exist_ok=True)
df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')

print(f"💾 Results saved to: {OUTPUT_CSV_FILE}")
print("\n🔍 Preview (First 5 rows):")
print(df.head().to_string())

📂 Loading data from: ../inputs/gold_standard_200.jsonl
✅ Loaded 200 test cases.
🚀 Starting RAG Generation for 200 queries...
--------------------------------------------------
[200/200] Processing ID: test_v2_00199...
✅ Completed processing 200 queries.
💾 Results saved to: ../outputs/rag_final_answers_200.csv

🔍 Preview (First 5 rows):
              id                                                                                                                                                                                                                                                                                                                                                                            query                                                                                                                                                                                                                                                                                     

---

In [2]:
import csv

# Đường dẫn file
csv_file_path = '../outputs/rag_final_answers_200.csv'

# Tạo iterator để đọc từng dòng
def read_csv_line_by_line(file_path, max_lines=50):
    """
    Iterator để đọc CSV file từng dòng một
    """
    with open(file_path, 'r', encoding='utf-8-sig') as f:
        csv_reader = csv.DictReader(f)
        
        for idx, row in enumerate(csv_reader):
            if idx >= max_lines:
                break
            yield idx + 1, row  # Trả về số dòng và nội dung

# Sử dụng iterator
print(f"📖 Đọc 50 dòng đầu tiên từ: {csv_file_path}")
print("=" * 80)

for line_num, row_data in read_csv_line_by_line(csv_file_path, max_lines=50):
    print(f"\n--- Dòng {line_num} ---")
    for key, value in row_data.items():
        # Giới hạn độ dài hiển thị để dễ đọc
        display_value = value[:300] + "..." if len(value) > 200 else value
        print(f"{key}: {display_value}")
    
print("\n" + "=" * 80)
print("✅ Hoàn thành!")

📖 Đọc 50 dòng đầu tiên từ: ../outputs/rag_final_answers_200.csv

--- Dòng 1 ---
id: test_v2_00000
query: Nhóm bạn thân từ chương trình Em Xinh Say Hi gồm những ai, và chuyến nghỉ dưỡng tại resort đắt đỏ ở Việt Nam có sự góp mặt của những người bạn nào ngoài nhóm chính, đồng thời họ đã cùng nhau tạo nên những khoảnh khắc nào đáng nhớ khiến cư dân mạng trầm trồ?...
answer: Nhóm bạn thân từ chương trình *Em Xinh Say Hi* gồm Bích Phương, Orange, Mỹ Mỹ và Quỳnh Anh Shyn.  

Chuyến nghỉ dưỡng tại resort đắt đỏ ở Việt Nam có sự góp mặt của các bạn bè ngoài nhóm chính: Tăng Duy Tân, Nam Phùng và producer Wokeup – bạn trai tin đồn của Orange.

Cả nhóm đã cùng nhau tạo nên nh...

--- Dòng 2 ---
id: test_v2_00001
query: Công ty TNHH Quốc tế HyunJin C&T đã bị xử phạt bao nhiêu tiền và phải thực hiện những biện pháp khắc phục hậu quả gì liên quan đến việc kinh doanh và quảng cáo sai lệch mỹ phẩm Désembre Derma Science High Frequency Cream Professional và DESEMBRE OXYJET ELIXIR TREATMENT, đồng thời 